In [ ]:
!pip install nsetools pandas nsepy PyPortfolioOpt bokeh

In [ ]:
from fetcher import Nifty500Fetcher

nff = Nifty500Fetcher()
nff.read_list(update=True)

In [ ]:
nff.fetch_data()

In [ ]:
nff.fetch_metadata()

In [ ]:
from cleaner import Nifty500Cleaner

nfc = Nifty500Cleaner()
nfc.clean_metadata()
nfc.clean_ohlc_data()

In [ ]:
from processor import StockProcessor

sp = StockProcessor('3MINDIA', meta_data_loc='data/cleaned/Metadata/nifty500.json')

In [ ]:
sp.process_metrics()

In [ ]:
sp.stock.metadata

In [ ]:
from processor import IndexProcessor

pp = IndexProcessor("data/cleaned/OHLC/Nifty500", 
                    "data/cleaned/Metadata/nifty500.json", 
                    "data/cleaned/Metadata/proc_nifty500.json")

In [ ]:
pp.process_metrics()

In [ ]:
import pandas as pd
pd.read_csv('data/cleaned/OHLC/Nifty500/M&M.csv')

In [ ]:
from processor import IndexProcessor

pp = IndexProcessor("data/cleaned/OHLC/Nifty500", "data/cleaned/Metadata/nifty500.json")
pp.process_close(30)

In [ ]:
from optimizer import EffOptimizer, HRPOptimizer, CLAOptimizer

opt1 = EffOptimizer(pp)
# opt2 = HRPOptimizer(pp)
# opt3 = CLAOptimizer(pp)

In [ ]:
opt1.optimize_max_sharpe()

In [ ]:
opt1.portfolio.statistics

In [ ]:
opt1.optimize_min_volatility()

In [ ]:
opt1.portfolio.statistics

In [ ]:
opt2 = HRPOptimizer(pp)
opt2.optimize()

In [ ]:
opt2.portfolio.statistics

In [ ]:
temp_df = (pp.close_matrix.pct_change(30).tail(1)*100 > 0)
temp_df

In [ ]:
opt3 = CLAOptimizer(pp)
opt3.optimize_max_sharpe()

In [ ]:
opt3.portfolio.statistics

In [ ]:
opt3.optimize_min_volatility()
opt3.portfolio.statistics

In [ ]:
import pandas as pd
df = pd.read_csv('data/raw/nifty_500_metadata.csv')#.to_dict(orient='records')

In [ ]:
some = df[df['symbol']=='M&amp;M'].to_dict(orient='records')

In [ ]:
some

In [ ]:
from fetcher import Nifty500Fetcher

nff = Nifty500Fetcher()
nff.fetch_index('NIFTY')

In [ ]:
from simulate import Backtesting, Agent
from optimizer import EffOptimizer, HRPOptimizer, CLAOptimizer
from datetime import date

agents = [
        Agent("CLA_MAX_SHARPE", 'navy', CLAOptimizer(), 1000000, optimizer_type='max_sharpe',
                            metadata_loc="data/processed/Metadata/nifty500.json", single_day_cash=0.80),
         Agent("EFF_MIN_VOL", 'green', EffOptimizer(), 1000000, optimizer_type='min_vol',
                           metadata_loc="data/processed/Metadata/nifty500.json", single_day_cash=0.80),
         Agent("EFF_MAX_SHARPE", 'dimgray', EffOptimizer(), 1000000, optimizer_type='max_sharpe',
                           metadata_loc="data/processed/Metadata/nifty500.json", single_day_cash=0.80),
         Agent("HRP", 'orangered', HRPOptimizer(), 1000000, metadata_loc="data/processed/Metadata/nifty500.json", 
                           single_day_cash=0.80),
         Agent("CLA_MIN_VOL", 'magenta', CLAOptimizer(), 1000000, optimizer_type='min_vol',
                           metadata_loc="data/processed/Metadata/nifty500.json", single_day_cash=0.80)]

benchmarks = [
    {
        'name':'NIFTY500',
        'data_loc':'data/raw/Nifty500/Index/NIFTY500.csv',
        'color':'red'
    },
]

bt = Backtesting(#end_date=date(2011, 12, 31), 
    proc_ohlc_loc="data/processed/OHLC/Nifty500", 
              proc_metadata_loc="data/processed/Metadata/nifty500.json", bband_margins=False,
                 lookback_period=1,rebalance_period=1,
                 bband_ma=20, benchmarks=benchmarks, #process_metrics=True,
                agents=agents)

In [ ]:
bt.cret_plot()

In [ ]:
bt.portfolio_plot()

In [ ]:
bt.backtest()

In [ ]:
import pandas as pd
df = bt.processor.close_matrix
#df[df.index <= date(2012, 5, 21)][['BHARATRAS']].tail()
df.index = pd.to_datetime(df.index)
df[df.index.year < 2015].tail()

In [ ]:
from datetime import date
bt.env.data[bt.env.data.index == date(2012, 6, 13)]['DEEPAKNTR']

In [ ]:
import pandas as pd
df = pd.read_csv('data/processed/OHLC/Nifty500/BHARATRAS.csv')
df.Date = pd.to_datetime(df.Date).dt.date
df[df.Date <= date(2012, 5, 21)].tail()